In [ ]:
import pathlib
import pandas as pd
import sns

csvfile = '/dls/labxchem/data/2018/lb18145-80/processing/analysis/eugene/panddaproject/training_data_paths/training_set.csv'

df = pd.read_csv(csvfile)



def num_remodelled_vs_not_all_systems(df):
    def num_remodelled_vs_not(system):
        df = df[df.system == system]
        df_remodelled = df[df['remodelled'] == 1]
        df_not_remodelled = df[df['remodelled'] == 0]
        df_remodelled_percentage = len(df_remodelled) / len(df)
        return {'system': system,
                'len_remodelled': len(df_remodelled), 
                'len_not_remodelled': len(df_not_remodelled),
                'remodelled_percentage': df_remodelled_percentage,
                'not_remodelled_percentage': 1 - df_remodelled_percentage}

    systems = df.system.unique()

    lst = list(map(num_remodelled_vs_not, systems))
    return lst

lst = num_remodelled_vs_not_all_systems(df)
data = pd.DataFrame(lst)

